In [1]:
versioninfo()

Julia Version 0.6.4
Commit 9d11f62bcb (2018-07-09 19:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-4500U CPU @ 1.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell MAX_THREADS=16)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


In [2]:
using StatsBase
using IterTools

In [56]:
Pkg.add("LightGraphs")

INFO: Cloning cache of LightGraphs from https://github.com/JuliaGraphs/LightGraphs.jl.git
INFO: Cloning cache of SimpleTraits from https://github.com/mauro3/SimpleTraits.jl.git
INFO: Installing LightGraphs v0.12.0
INFO: Installing SimpleTraits v0.6.0
INFO: Building CodecZlib
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of LightGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
function evaluate(solution)
    objective_function = 0
    
    for i in 1:(size(solution)[1]-1)
        objective_function += distances[solution[i],solution[i+1]]
    end

    objective_function += distances[solution[end],solution[1]]
    
    return objective_function
end

function first_sequential_k_opt2(solution)
    #solution_ref = copy(solution)
    #push!(solution_ref, solution[end])
    
    #search
    for i in 1:length(solution)-1
        used_cities = []
        old_weights = distances[solution[i],solution[i+1]]
        new_gain = 0
        
        new_solution = vcat(solution[i:-1:1], solution[end:-1:i+1])
        
        push!(used_cities, solution[i])
        push!(used_cities, solution[i+1])
        for k = 2:5
            found = false
            for city in near_cities[new_solution[end]]
                candidate = findin(new_solution, city)
                if length(candidate) == 1
                    j = candidate[1]
                    if ~(city in used_cities) & ~(new_solution[j+1] in used_cities)
                        if ((old_weights + distances[new_solution[j],new_solution[j+1]]) >
                            (new_gain + distances[new_solution[end], new_solution[j]]
                            + distances[new_solution[1], new_solution[j+1]]))
                            
                            found = true
                            
                            old_weights += distances[new_solution[j],new_solution[j+1]]
                            new_gain += distances[new_solution[end],new_solution[j]]
                            
                            push!(used_cities, new_solution[j])
                            push!(used_cities, new_solution[j+1])
                            
                            new_solution = vcat(new_solution[1:j], new_solution[end:-1:j+1])
                            
                            break
                        end

                    end
                end
            end
            if found == false
                break
            end
        end
        if length(used_cities) > 2
            return new_solution, true
        end 
    end
    #@show solution
    return solution, false 
end

first_sequential_k_opt2 (generic function with 1 method)

In [5]:
#time_limit = Dates.now() + Dates.Second(300)#+ Dates.Second(0.1) #+ Dates.Minute(5)

m = readdlm("TSP_instances/problem_500_100_10000_10000.pctsp");

In [100]:
@time begin
    
n_cities = 40
n_cities_half_up = Int(ceil((n_cities + 1)/2))
n_cities_half_down = Int(floor((n_cities + 1)/2))

init = rand(1:80) # 1 ~ 90

distances = m[(2+init):(init+n_cities+1), init:(init+n_cities-1)]

end

  0.000059 seconds (12 allocations: 12.938 KiB)


40×40 Array{Float64,2}:
    0.0  1488.0  7535.0  8143.0  5327.0  …  7770.0   238.0  7724.0  7042.0
 1488.0     0.0  2121.0  1135.0  8011.0     3256.0  4633.0  4149.0  6493.0
 7535.0  2121.0     0.0  1613.0  2017.0     3956.0  8729.0  3771.0  9163.0
 8143.0  1135.0  1613.0     0.0  3950.0     8495.0  7998.0  7577.0   407.0
 5327.0  8011.0  2017.0  3950.0     0.0     3232.0  6457.0  6015.0  4325.0
 3254.0  3625.0  1133.0  7161.0  5140.0  …  5967.0  6680.0  2201.0  8938.0
  495.0  6446.0  3735.0  2102.0  1152.0     4065.0  7008.0   726.0  1513.0
 8896.0  1556.0  9817.0  3560.0  6473.0        6.0  6150.0  3785.0  1259.0
 3569.0   944.0  1660.0  8241.0  2333.0       77.0  9255.0   519.0  8261.0
 4215.0  9419.0  8336.0  7392.0  4125.0     9628.0  2347.0  5898.0  8413.0
 6546.0  6263.0  2388.0  2007.0   645.0  …  5272.0  6844.0  6360.0  9352.0
 5466.0  1997.0  7755.0  6964.0   705.0     5797.0  9957.0  3771.0  3970.0
 9720.0  5021.0  4333.0  8543.0  5315.0     7518.0  2785.0  6780.0  1230.0
 

In [111]:
centers = readdlm("routing_files/centers.csv")

15×1 Array{Any,2}:
 "8.000000000000000000e+01,1.036666666666666714e+02"
 "1.226875000000000000e+02,4.418750000000001421e+01"
 "1.170000000000000000e+02,1.368333333333333428e+02"
 "2.833333333333332860e+01,1.071666666666666714e+02"
 "7.563636363636362603e+01,7.309090909090909349e+01"
 "1.286666666666666572e+02,8.583333333333332860e+01"
 "8.607142857142856940e+01,9.042857142857143060e+01"
 "1.402777777777777715e+02,5.927777777777779278e+01"
 "1.364444444444444287e+02,1.221111111111111143e+02"
 "1.013333333333333286e+02,1.586666666666666856e+02"
 "9.122222222222222854e+01,1.352222222222222285e+02"
 "6.299999999999999289e+01,1.315000000000000000e+02"
 "9.785714285714286120e+01,5.128571428571429891e+01"
 "1.196250000000000000e+02,1.086250000000000000e+02"
 "1.045000000000000000e+02,7.250000000000000000e+01"

In [112]:
centers = readdlm("routing_files/centers.csv")
clusters = readdlm("routing_files/clusters.csv")
euclidean_distances = readdlm("routing_files/euclidean_distances.csv")
coord_of_POI = readdlm("routing_files/coord_of_POI.csv")

142×1 Array{Any,2}:
 "2.800000000000000000e+01,1.060000000000000000e+02"
 "2.800000000000000000e+01,1.070000000000000000e+02"
 "2.800000000000000000e+01,1.080000000000000000e+02"
 "2.800000000000000000e+01,1.090000000000000000e+02"
 "2.900000000000000000e+01,1.060000000000000000e+02"
 "2.900000000000000000e+01,1.070000000000000000e+02"
 "5.200000000000000000e+01,1.220000000000000000e+02"
 "6.400000000000000000e+01,1.330000000000000000e+02"
 "6.500000000000000000e+01,1.330000000000000000e+02"
 "6.500000000000000000e+01,1.340000000000000000e+02"
 "6.600000000000000000e+01,1.330000000000000000e+02"
 "6.600000000000000000e+01,1.340000000000000000e+02"
 "7.300000000000000000e+01,7.300000000000000000e+01"
 ⋮                                                  
 "1.370000000000000000e+02,1.100000000000000000e+02"
 "1.370000000000000000e+02,1.270000000000000000e+02"
 "1.370000000000000000e+02,1.280000000000000000e+02"
 "1.370000000000000000e+02,1.290000000000000000e+02"
 "1.380000000000000000e+02

In [109]:
@time begin

#m=0

max_near_cities = 5

near_cities = Dict()
    
end

# STEP 1 - preprocessing nearest cities for heuristic

@time for i in 1:n_cities
    near_cities[i] = selectperm(distances[i, :], 1:max_near_cities+1)
    shift!(near_cities[i])
end

# STEP 2 - UPPERBOUND (quasi) Lin Kernighan - First improvement sequential 5-opt
@time begin
    upper_bound = 1e10
    a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
    best_heuristic_solution = []
    
    for i in 1:500*(n_cities)
        a, better  = first_sequential_k_opt2(a)
        if better == false
            value = evaluate(a)
            if value < upper_bound
                upper_bound = value
                best_heuristic_solution = copy(a)
            end
            a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
        end
    end

    value = evaluate(a)
    if value < upper_bound
        upper_bound = value
    end
end

heuristic_upper_bound = upper_bound

  0.000022 seconds (8 allocations: 768 bytes)
  0.000173 seconds (546 allocations: 57.375 KiB)
  1.201516 seconds (8.90 M allocations: 597.343 MiB, 14.67% gc time)


14216.0